# 📘 Enzyme Kinetics: Michaelis-Menten Analysis> Fit Michaelis-Menten model to enzyme velocity data⏱️ **15-25 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Biology** | 🔬 **Biochemistry**---

## 🔬 Domain Background**Model:** $v = \frac{V_{max}[S]}{K_m + [S]}$**Applications:** Enzyme characterization, drug screening---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import numpy as npimport jax.numpy as jnpfrom nlsq import curve_fitimport matplotlib.pyplot as pltdef michaelis_menten(S, Vmax, Km):    return Vmax * S / (Km + S)

## Generate Data

In [ ]:
Vmax_true, Km_true = 100, 50S = np.array([1,2,5,10,20,50,100,200,500])v_true = michaelis_menten(S, Vmax_true, Km_true)v = v_true + np.random.normal(0, 3, len(S))plt.plot(S, v, 'o')plt.xlabel('[S] (µM)')plt.ylabel('v (µM/s)')plt.tight_layout()
display(fig)
plt.close(fig)


## Fit Model

In [ ]:
popt, pcov = curve_fit(michaelis_menten, S, v, p0=[80, 40])Vmax_fit, Km_fit = poptprint(f'Vmax: {Vmax_fit:.1f} µM/s')print(f'Km: {Km_fit:.1f} µM')

## Key Insights1. **Km indicates affinity** - Lower Km = higher substrate affinity2. **Vmax depends on enzyme amount**---